In [96]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import re
import yfinance
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt 
from datetime import datetime

yfinance.pdr_override()

# Get html code of website that has stock list for S&P 500
url = 'https://www.slickcharts.com/sp500'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, 'html')

# Find the table holding the list of stocks
table = soup.find('table')
temp = pd.read_html(table.prettify())
sp = pd.DataFrame(temp[0])

# Put the stock symbols into one list
stockList = []
for idx, row in sp.iterrows():
    sym = row['Symbol']
    sym = sym.replace('.','-')
    stockList.append(sym)

# Set up time objects
now = dt.datetime.now()
start = dt.datetime(now.year, now.month-2, now.day)

testStock = stockList[0]
print(testStock)
df = pdr.get_data_yahoo(testStock,start,now)

# Get rid of columns that are not of use and only taking up space
df.drop('Open', inplace=True, axis=1)
df.drop('Close', inplace=True, axis=1)
df.drop('Volume', inplace=True, axis=1)

# Create column of rolling highs and lows (High/Low over past 10 days)
df['L10'] = df['Low'].rolling(window=10).min()
df['H10'] = df['High'].rolling(window=10).max()

    # Calculate %K and %D (part of stochastic measure)
df['%K'] = 100*((df['Adj Close'] - df['L10']) / (df['H10'] - df['L10']))
df['%D'] = df['%K'].rolling(window=4).mean()

# Column to determine when to buy the stock 
# (if %K crosses through %D line and %D < 20)
df['Buy Entry'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < 25)

# Column to determine when to sell the stock
# (trailing stop: if position drops 8%)
s = df.loc[df['Buy Entry'].cummax().astype(bool), 'Adj Close'].cummax().pct_change().add(1).fillna(1)
df['Trailing Stop'] = s.clip(lower=1).cumprod()*.92*df.loc[s.index[0]]['Adj Close']
df['Buy Exit'] = (df['Trailing Stop'] > df['Adj Close'])

# Placeholder column for 1 when holding stock and 0 for not 
df['Long'] = np.nan
df.loc[df['Buy Entry'], 'Long'] = 1
df.loc[df['Buy Exit'], 'Long'] = 0
# Set initial long to 0
df['Long'][0] = 0
# Fill empty spots of long
df['Long'] = df['Long'].fillna(method='pad')
# The return if we just had the stock the whole time
df['Straight Stock'] = 100 * (df['Adj Close'] - df.iloc[0]['Adj Close']) / df.iloc[0]['Adj Close']
# The return if we bought using this strategy
df['Strategy Returns'] = (100*(df['Adj Close'] - df.loc[s.index[0]]['Adj Close']) / df.loc[s.index[0]]['Adj Close']) * df['Long'].shift(1)

print(df)

# Go through each stock in list and test the purchasing setup
#for stock in stockList:
 #   df = pdr.get_data_yahoo(stock,start,now)

    # Create column of rolling highs and lows (High/Low over past 10 days)
#  df['L10'] = df['Low'].rolling(window=10).min()
 #   df['H10'] = df['High'].rolling(window=10).max()

    # Calculate %K and %D (part of stochastic measure)
  #  df['%K'] = 100*((df['Close'] - df['L10']) / (df['H10'] - df['L10']))
   # df['%D'] = df['%K'].rolling(window=4).mean()



AAPL
[*********************100%***********************]  1 of 1 completed
                  High         Low   Adj Close         L10         H10  \
Date                                                                     
2020-10-12  125.180000  119.279999  124.185753         NaN         NaN   
2020-10-13  125.389999  119.650002  120.891434         NaN         NaN   
2020-10-14  123.029999  119.620003  120.981277         NaN         NaN   
2020-10-15  121.199997  118.150002  120.502106         NaN         NaN   
2020-10-16  121.550003  118.809998  118.815010         NaN         NaN   
2020-10-19  120.419998  115.660004  115.780251         NaN         NaN   
2020-10-20  118.980003  115.629997  117.307617         NaN         NaN   
2020-10-21  118.709999  116.449997  116.668724         NaN         NaN   
2020-10-22  118.040001  114.589996  115.550644         NaN         NaN   
2020-10-23  116.550003  114.279999  114.841873  114.279999  125.389999   
2020-10-26  116.550003  112.879997  11